In [29]:
import pandas as pd
import numpy as np

In [30]:
LSAS = pd.read_csv("../result/fstat/LSAS.csv")
# Set the starting voxel
start_voxel = 10211
end_voxel = 13718

# Generate random data
random_data = {
    'voxel': np.arange(start_voxel, end_voxel + 1),
    'f_stat': np.random.rand(end_voxel - start_voxel + 1),
    'p_val': np.random.rand(end_voxel - start_voxel + 1)
}

# Create a DataFrame for the random data
random_df = pd.DataFrame(random_data)

# Append the random data to the original DataFrame
LSAS = pd.concat([LSAS, random_df], ignore_index=True)

In [31]:
voxel_3d_index = pd.read_csv("../template/common_mask_as_np_coords.csv")

In [32]:
merged_df = pd.merge(LSAS, voxel_3d_index, left_on='voxel', right_index=True, how='left')

In [33]:
merged_df

,voxel,f_stat,p_val,x,y,z
0,0,1.276166,0.050933,2,18,15
1,1,1.120305,0.201667,2,18,16
2,2,1.013119,0.691000,2,19,14
3,3,0.997626,0.697333,2,19,15
4,4,1.093722,0.255733,2,19,16
...,...,...,...,...,...,...
13714,13714,0.634875,0.615977,39,27,17
13715,13715,0.422765,0.207911,39,27,19
13716,13716,0.598215,0.714257,39,27,20
13717,13717,0.460482,0.059118,39,27,21


In [50]:
import numpy as np
import pandas as pd
from nilearn import masking, plotting, image
from nilearn.image import resample_to_img, new_img_like
import os

def get_subject_codes():
    source_dir = "/mnt/NAS2/data/SAD_gangnam_resting/fMRIPrep/"
    subfolders = [name for name in os.listdir(source_dir) if os.path.isdir(os.path.join(source_dir, name))]
    filtered_subfolders = [code for code in subfolders if code.startswith('sub-')]
    return filtered_subfolders

def load_fmri(subject_code):
    fmri_file_path = f'/mnt/NAS2/data/SAD_gangnam_resting/fMRIPrep/{subject_code}/ses-01/func/{subject_code}_ses-01_task-rest_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz'
    fmri_img = image.load_img(fmri_file_path)
    return fmri_img


# 회색질 마스크 로드 및 이진화
gray_matter_mask_path = "/home/changbae/fmri_project/MDMR/template/tpl-MNI152NLin2009cAsym_space-MNI_res-01_class-GM_probtissue.nii"
gray_matter_mask = image.load_img(gray_matter_mask_path)

# 예제 fMRI 이미지를 로드하여 동일한 해상도로 재샘플링
subject_codes = get_subject_codes()
fmri = load_fmri(subject_codes[0])
example_fmri_img = image.load_img(fmri)

resampled_gray_matter_mask = resample_to_img(gray_matter_mask, example_fmri_img, interpolation='nearest')
gray_matter_mask_data = resampled_gray_matter_mask.get_fdata()
binary_gray_matter_mask_data = (gray_matter_mask_data > 0.5).astype(np.int32)
binary_gray_matter_mask = new_img_like(resampled_gray_matter_mask, binary_gray_matter_mask_data)

# f-statistics를 위한 빈 3D 배열 초기화
f_stat_map_data = np.zeros(binary_gray_matter_mask.shape)

# 대응하는 값들로 3D 배열 채우기
for idx, row in LSAS.iterrows():
    x, y, z = voxel_3d_index.loc[idx, ['x', 'y', 'z']]
    f_stat_map_data[int(x), int(y), int(z)] = row['f_stat']

# 샘플 f-stat 값 확인
print(f"Sample f-stat values: {f_stat_map_data[f_stat_map_data > 0][:10]}")

# 3D 배열에서 새로운 이미지 객체 생성
f_stat_map_img = new_img_like(binary_gray_matter_mask, f_stat_map_data)

# f-stat 맵의 값 범위 확인
print(f"f-stat map min: {f_stat_map_data.min()}, max: {f_stat_map_data.max()}")

# f-statistics 맵을 인터랙티브하게 시각화
html_view_f_stat = plotting.view_img(f_stat_map_img, threshold=0, vmax=1.8, 
                                     bg_img=gray_matter_mask, cut_coords=[0, 0, 0], 
                                     title="F-statistics Map", colorbar=True)
html_view_f_stat.open_in_browser()

Sample f-stat values: [1.27616638 1.12030492 1.01311896 0.99762566 1.09372224 1.00628055
 1.02513683 0.9431508  1.20081975 1.0004717 ]
f-stat map min: 0.0, max: 1.836541497666757


/home/changbae/anaconda3/envs/fmrienv/lib/python3.9/site-packages/numpy/_core/fromnumeric.py:820: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)


In [52]:
# 회색질 마스크 로드 및 이진화
gray_matter_mask_path = "/home/changbae/fmri_project/MDMR/template/tpl-MNI152NLin2009cAsym_space-MNI_res-01_class-GM_probtissue.nii"
gray_matter_mask = image.load_img(gray_matter_mask_path)

# 예제 fMRI 이미지를 로드하여 동일한 해상도로 재샘플링
subject_codes = get_subject_codes()
fmri = load_fmri(subject_codes[0])
example_fmri_img = image.load_img(fmri)

resampled_gray_matter_mask = resample_to_img(gray_matter_mask, example_fmri_img, interpolation='nearest')
gray_matter_mask_data = resampled_gray_matter_mask.get_fdata()
binary_gray_matter_mask_data = (gray_matter_mask_data > 0.5).astype(np.int32)
binary_gray_matter_mask = new_img_like(resampled_gray_matter_mask, binary_gray_matter_mask_data)

# p-value를 위한 빈 3D 배열 초기화
p_val_map_data = np.zeros(binary_gray_matter_mask.shape)

# 대응하는 값들로 3D 배열 채우기
for idx, row in LSAS.iterrows():
    x, y, z = voxel_3d_index.loc[idx, ['x', 'y', 'z']]
    p_val_map_data[int(x), int(y), int(z)] = row['p_val']

# 샘플 p-value 값 확인
print(f"Sample p-value values: {p_val_map_data[p_val_map_data > 0][:10]}")

# 3D 배열에서 새로운 이미지 객체 생성
p_val_map_img = new_img_like(binary_gray_matter_mask, p_val_map_data)

# p-value 맵의 값 범위 확인
print(f"p-value map min: {p_val_map_data.min()}, max: {p_val_map_data.max()}")

# p-value < 0.05인 영역만 시각화하기 위해 임계값 적용
thresholded_p_val_map_data = np.zeros_like(p_val_map_data)
thresholded_p_val_map_data[p_val_map_data < 0.05] = p_val_map_data[p_val_map_data < 0.05]

# 3D 배열에서 새로운 임계값이 적용된 이미지 객체 생성
thresholded_p_val_map_img = new_img_like(p_val_map_img, thresholded_p_val_map_data)

# 임계값이 적용된 p-value 맵을 인터랙티브하게 시각화
html_view_p_val = plotting.view_img(thresholded_p_val_map_img, threshold=0, vmax=0.05, 
                                     bg_img=gray_matter_mask, cut_coords=[0, 0, 0], 
                                     title="P-value Map (Thresholded at 0.05)", colorbar=True)
html_view_p_val.open_in_browser()

Sample p-value values: [0.05093333 0.20166667 0.691      0.69733333 0.25573333 0.71833333
 0.60133333 0.98533333 0.09486667 0.76033333]
p-value map min: 0.0, max: 0.9998053516689113


/home/changbae/anaconda3/envs/fmrienv/lib/python3.9/site-packages/numpy/_core/fromnumeric.py:820: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)
